In [2]:
import random as random
import Bio
import numpy as np
import sys
import os
import networkx as nx
import random as random
from tqdm import tqdm # loading bar
from utils_copy import linear_C, get_cost_2, get_sequence_string, parse_fasta_multiple, create_score_matrix, write_alignments_to_file, linear_backtrack, fill_graph,new_sp_approxi_combi
from utils_copy import convert_to_desired_format_nr_version, compute_cost, my_traversal_simply, extend_alignment_chaos, find_min_span_edges_testing, parse_fasta_multiple_remove_n
import timeit
from utils_copy import al_integrity_testt
from old_for_testing.sp_approx import sp_approx
from old_for_testing.utils import *


score_matrix={'a': {'a': 0, 'c': 5, 'g': 2, 't': 5}, 'c': {'a': 5, 'c': 0, 'g': 5, 't': 2}, 'g': {'a': 2, 'c': 5, 'g': 0, 't': 5}, 't': {'a': 5, 'c': 2, 'g': 5, 't': 0}}
gap_cost=5

In [3]:
def multiple_merge_algo(seqs: list[str], score_matrix: dict, gap_cost: int, verbose=False, return_center_string=False,layout="spring"):
    # Make a matrix to hold pairwise alignment costs for all alignment combinations!
    matrix = np.full((len(seqs), len(seqs)), np.nan)
    # Loop over all pairs
    for i, seq1 in enumerate(seqs):
        for j, seq2 in enumerate(seqs):
            matrix[i, j] = get_cost_2(linear_C(gap_cost, score_matrix, seq1, seq2,verbose=verbose))
    if verbose:
        print("Here comes the distance matrix produced by the alignments: \n")
        print(matrix)
    matrix_for_MST=matrix #copy the matrix, so that we can keep the old matrix and make a changed version to the "pseudomatrix" version
    matrix_for_MST=convert_to_desired_format_nr_version(matrix_for_MST) #making the "pseudomatrix"
    TODO: #till here I'd say we need no changes!!
    min_span_edges=find_min_span_edges_testing(matrix_for_MST,verbose=verbose) #Run Kruskal's algorithm on the "pseudomatrix"
    if verbose:
        print("Here comes the pseudomatrix, filled out with with the edges inclued in the MST: \n")
        print(min_span_edges)

    max_indices = np.where(matrix == np.max(matrix)) # Choosing where to start traversal. I want to start at one of the nodes that is the furthest away from any other so start from a side of graph.. hmmmm...
    max_row_index = max_indices[0][0] #just choose one of them.
    if verbose:
        print("Starting key for traversal based on max_row_idex: ")
        print(max_row_index)

    #Put the nodes and the minimum spanning edges into a graph.
    G=fill_graph(min_span_edges,str(int(max_row_index)),layout) #using the max_row_index as the starting key! (an making the graph!)
    alignment_pairs,index_dict=my_traversal_simply(G,str(int(max_row_index)),verbose=verbose) #'traverse' to get alignment_pairs (pairs of sucessors and predecessors) and their position in the MSA to come (index_dict)
    if verbose:
        print("Here come your alignment pairs and the idex dict: \n")
        print(alignment_pairs)
        print(index_dict)

     # Constructing alignment M
    M: list[list[str]] = [[letter] for letter in [*seqs[int(max_row_index)]]] #make structure where evey column in the alignment is represented as a string in a list (in a list)
    cost_list = []
    A_dict={}
    #using the pairings of predecessors and successors in the alignment_pairs dict, align the strings.
    for key,value in alignment_pairs.items():
        if verbose: print("this is the key: "+str(key)+" and this is the value: "+str(value))
        cost = linear_C(gap_cost, score_matrix, seqs[int(value)], seqs[int(key)],verbose=verbose) #the alignment call itself :) 
        if verbose: print("\n now aligning...."+str(seqs[int(key)])+ " and "+ str(seqs[int(value)]))
        cost_list.append(get_cost_2(cost))
        
        # prepare A-matrix for extension
        alignment1_str, alignment2_str = linear_backtrack(seqs[int(value)], seqs[int(key)], cost, score_matrix, gap_cost,verbose=verbose) #backtract to get the alignments!
        str1_nr=value #the predecessor/parent string
        alignment1, alignment2 = [*alignment1_str], [*alignment2_str] #splitting up the alignments into elements to have the right format for the list of lists (M)
        
        A = [list(e) for e in zip(alignment1,alignment2)] #zipping the elements of the two aligned strings together pairwisely
        if ord(key) < ord(value):
            key_for_A_dict=str(key)+"_"+str(value)
        else:
            key_for_A_dict=str(value)+"_"+str(key)
        A_dict[key_for_A_dict]=A
        #pair_al=A_dict[key_for_A_dict]
        if verbose: print("A right now is: "+str(A))
        if verbose: print("M right now: "+str(M))
        # extend
        Mk = extend_alignment_chaos(M,str1_nr, A,index_dict,verbose=verbose) 
        M = Mk
    if verbose:
        print("Here is the alignment in full omg: \n")
        print(M)
    print("this is A_dict: ")
    print(A_dict)
    al_integrity_testt(A,alignment_pairs,index_dict,M,verbose=verbose)
    # ACTUALLY COMPUTE (approximate) COST
    total_cost = compute_cost(M, score_matrix, gap_cost)
    print("Total cost of MSA:"+str(total_cost))
    return total_cost, M, matrix_for_MST,G

SyntaxError: invalid syntax (2203443271.py, line 13)

In [11]:
test_res_mat=np.array([['*', '1', '0', '1'], ['*', '2', '0', '2'],['','3','0','2'], ['', '3', '1', '2']])

In [19]:
def orchestrate_merge(res_matrix, seqs,score_matrix,gapcost,verbose=False): #should also include seqs,score_matrix,gap_cost, verbose=False)
    in_which_MSA_is_it={}
    names= np.unique(res_matrix[:, 2:])
    in_which_MSA_is_it ={name: [int(name),0] for name in names}
    MSA_list=[[*seq] for seq in seqs] #initalizing it as every string in their own alignment with nobody else :) 
    print(in_which_MSA_is_it)
    for row in res_matrix:
        if row[0]=="*":
            node1 = row[2]  # Extract the first node
            node2 = row[3]  # Extract the second node
            cost=linear_C(gap_cost,score_matrix,seqs[node1],seqs[node2]) #doing the alignment of the two strings :) 
            alignment1_str,alignment2_str=linear_backtrack(seqs[int(node1)], seqs[int(node2)], cost, score_matrix, gap_cost,verbose=verbose)
            alignment1, alignment2 = [*alignment1_str], [*alignment2_str]
            A = [list(e) for e in zip(alignment1,alignment2)] #the 'guide' (pairwise alignment) in this iteration!
            #where are these two sequences now? get the MSA and their position in that MSA!
            which_MSA_node1=in_which_MSA_is_it[node1][0]
            which_pos_in_MSA_node1=in_which_MSA_is_it[node1][1]
            which_MSA_node2=in_which_MSA_is_it[node2][0]
            which_pos_in_MSA_node2=in_which_MSA_is_it[node2][1]
            current_MSA_node1=MSA_list[which_MSA_node1][which_pos_in_MSA_node1] #how is the current MSA of node 1 looking??
            current_MSA_node2=MSA_list[which_MSA_node2][which_pos_in_MSA_node2] #how is the current MSA of node 1 looking??
            #now we should compare the A (pairwise alignment) to the current alignment
            MA=[]#for holding the merged stuff
            i=0
            j=0
            while i < len(current_MSA_node1) and j < len(A):
                #case1
                if current_MSA_node1[i][which_pos_in_MSA_node1]=='-' and A[j][0]=='-':
                    current_MSA_node1[i].append(A[j][1])
                    MA.append(current_MSA_node1[i])
                    i+=1
                    j+=1
                #case2
                elif current_MSA_node1[i][which_pos_in_MSA_node1]=='-' and A[j][0]!='-':
                    current_MSA_node1[i].append('-')
                    MA.append(current_MSA_node1[i])
                    i+=1
                #case3
                elif current_MSA_node1[i][which_pos_in_MSA_node1]!='-' and A[j][0]=='-':
                    c = ['-']*len(current_MSA_node1[i])
                    c.append(A[j][1])
                    MA.append(c)
                    j+=1
                #case4
                elif current_MSA_node1[i][which_pos_in_MSA_node1]!='-' and A[j][0]!='-':
                    current_MSA_node1[i].append(A[j][1])
                    MA.append(current_MSA_node1[i])
                    i = i + 1
                    j = j + 1
            if i < len(current_MSA_node1):
                while i < len(current_MSA_node1):
                    current_MSA_node1[i].append('-')
                    MA.append(current_MSA_node1[i])
                    i = i + 1
            
            if j < len(A):
                k = len(current_MSA_node1[which_pos_in_MSA_node1])
                while j < len(A):
                    c = ['-']*(k-1)
                    c.append(A[j][1])
                    MA.append(c)
                    j = j + 1
            




merge_actually() #TODO: #um actually make this lollll




In [28]:
names=np.unique(test_res_mat[:, 2:])
print(names)
#in_which_MSA_is_itt = {name:[int(name):0 for i, name in enumerate(names)]}
in_which_MSA_is_itt = {name: [int(name),0] for name in names}
print(in_which_MSA_is_itt)

seqss=['agct','ctgg','gtacccacaca']
MSA_list=[[*seq] for seq in seqss]
print(MSA_list)


['0' '1' '2']
{'0': [0, 0], '1': [1, 0], '2': [2, 0]}
[['a', 'g', 'c', 't'], ['c', 't', 'g', 'g'], ['g', 't', 'a', 'c', 'c', 'c', 'a', 'c', 'a', 'c', 'a']]


In [20]:
orchestrate_merge(test_res_mat)

{0: {'0': 0}, 1: {'1': 0}, 2: {'2': 0}}